<a href="https://colab.research.google.com/github/YixiaoHong/Finencial-Engineering-P1/blob/master/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finencial Engineering Computational Project

## Yixiao Hong - 1001311145
## Hongzheng Xu - 1001772904

In [0]:
#Import libraries needed for the project:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

!pip install qpsolvers

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
import scipy as sp

# Problem a)

## 1.0 Data Importing and Cleaning

In [0]:
stocksNameList = ['F','CAT','DIS','MCD','KO','PEP','WMT','C','WFC','JPM','AAPL','IBM','PFE','JNJ','XOM','MRO','ED','T','VZ','NEM']
print('Stock names:',stocksNameList)
print(len(stocksNameList))

Stock names: ['F', 'CAT', 'DIS', 'MCD', 'KO', 'PEP', 'WMT', 'C', 'WFC', 'JPM', 'AAPL', 'IBM', 'PFE', 'JNJ', 'XOM', 'MRO', 'ED', 'T', 'VZ', 'NEM']
20


In [0]:
df =pd.read_csv('https://raw.githubusercontent.com/YixiaoHong/Finencial-Engineering-P1/master/data/AAPL.csv')
df_stock_price_full = df[['Date']]

for i in stocksNameList:
  path = 'https://raw.githubusercontent.com/YixiaoHong/Finencial-Engineering-P1/master/data/' + i +'.csv'
  df =pd.read_csv(path)
  df_stock_price_full[i] = df[['Adj Close']]
print (df_stock_price_full.shape)

(48, 21)


In [0]:
df_stock_price_full['Date'] = df_stock_price_full['Date'].astype('datetime64[ns]') 
display(df_stock_price_full)

,Date,F,CAT,DIS,MCD,KO,PEP,WMT,C,WFC,JPM,AAPL,IBM,PFE,JNJ,XOM,MRO,ED,T,VZ,NEM
0,2005-01-01,8.607593,28.674561,23.239105,20.590382,10.552325,35.125565,37.238281,437.080566,18.352392,24.607668,4.755152,62.059471,13.035284,41.500927,32.743988,6.852079,22.034508,10.387199,14.897937,33.243374
1,2005-02-01,8.330702,30.859501,22.679024,21.029022,10.885496,35.230228,36.676849,425.229126,17.777571,24.304571,5.547884,61.501465,14.184512,42.078224,40.174839,8.375353,21.471962,10.653401,15.219813,35.969017
2,2005-03-01,7.461411,29.687452,23.320276,19.795757,10.598103,34.687309,35.610874,400.811768,18.185806,23.007877,5.153373,60.820072,14.279902,43.265335,38.005806,8.487700,21.459938,10.489573,15.020938,33.770912
3,2005-04-01,5.999423,28.586847,21.429001,18.632418,11.193820,36.550690,33.598995,418.827728,18.228388,23.599710,4.459579,50.836479,14.769124,44.212318,36.366962,8.424388,22.019592,10.538280,15.147880,30.417458
4,2005-05-01,6.642731,30.841572,22.273172,19.668612,11.500459,36.957958,33.663162,420.558899,18.371319,24.010963,4.917161,50.284065,15.165947,43.226677,35.837692,8.771710,23.154150,10.492134,15.161038,29.832663
5,2005-06-01,6.815792,31.234846,20.438723,17.640724,10.758329,35.427357,34.464516,412.702881,19.030146,23.722153,4.552333,49.516693,15.095690,42.079090,36.834179,9.858621,24.133881,10.658173,14.805373,31.352509
6,2005-07-01,7.148591,35.334606,20.812099,19.814829,11.421812,35.988895,35.286800,388.331696,18.955980,23.601250,5.274571,55.696301,14.504559,41.405819,37.654568,10.780385,24.814003,10.972307,14.668242,30.163643
7,2005-08-01,6.697522,36.541264,20.446838,20.628532,11.484452,36.200085,32.147804,391.129791,18.424444,22.982948,5.798935,53.801048,13.946273,41.036823,38.391640,11.879480,24.169954,10.952836,14.200024,31.794313
8,2005-09-01,6.623626,38.688038,19.586428,21.289656,11.273030,37.427654,31.431786,406.767883,18.410175,23.010078,6.630004,53.662868,13.765078,41.177101,40.924217,12.959023,25.318768,10.902798,14.191338,37.891315
9,2005-10-01,5.589106,34.631557,19.781240,20.088177,11.309140,39.176384,33.935135,409.091125,18.922527,24.834333,7.122217,54.773315,11.984496,40.747635,36.158108,11.310238,23.728195,10.848213,13.679079,34.304070


In [0]:
#we only keep the data from Dec-30-2004 to Sep-30-2008 (Oct-1-2008) as the sample
df_stock_price_sample_data = df_stock_price_full.iloc[:-2,:].copy()
print (df_stock_price_sample_data.shape)

(46, 21)


In [0]:
#compute the monthly return rate
df_monthly_r = df_stock_price_full.copy()
for row in range(47):
  for col in range(1,21):
    r = (df_monthly_r.iloc[row+1,col]-df_monthly_r.iloc[row,col])/df_monthly_r.iloc[row,col]
    df_monthly_r.iat[row,col] = r

#delete the last row+
df_monthly_r.drop(df_monthly_r.tail(1).index,inplace=True)

df_monthly_r_full_data = df_monthly_r.drop(columns=['Date'])

display(df_monthly_r_full_data)

,F,CAT,DIS,MCD,KO,PEP,WMT,C,WFC,JPM,AAPL,IBM,PFE,JNJ,XOM,MRO,ED,T,VZ,NEM
0,-0.032168,0.076198,-0.024101,0.021303,0.031573,0.002980,-0.015077,-0.027115,-0.031321,-0.012317,0.166710,-0.008991,0.088163,0.013910,0.226938,0.222308,-0.025530,0.025628,0.021605,0.081991
1,-0.104348,-0.037980,0.028275,-0.058646,-0.026401,-0.015411,-0.029064,-0.057422,0.022963,-0.053352,-0.071110,-0.011079,0.006725,0.028212,-0.053990,0.013414,-0.000560,-0.015378,-0.013067,-0.061111
2,-0.195940,-0.037073,-0.081100,-0.058767,0.056210,0.053719,-0.056496,0.044949,0.002341,0.025723,-0.134629,-0.164150,0.034259,0.021888,-0.043121,-0.007459,0.026079,0.004643,0.008451,-0.099300
3,0.107228,0.078873,0.039394,0.055612,0.027394,0.011143,0.001910,0.004133,0.007841,0.017426,0.102607,-0.010866,0.026868,-0.022293,-0.014554,0.041228,0.051525,-0.004379,0.000869,-0.019226
4,0.026053,0.012751,-0.082361,-0.103103,-0.064530,-0.041415,0.023805,-0.018680,0.035862,-0.012028,-0.074195,-0.015261,-0.004633,-0.026548,0.027806,0.123911,0.042313,0.015825,-0.023459,0.050946
5,0.048828,0.131256,0.018268,0.123244,0.061672,0.015850,0.023859,-0.059053,-0.003897,-0.005097,0.158652,0.124798,-0.039159,-0.016000,0.022272,0.093498,0.028181,0.029474,-0.009262,-0.037919
6,-0.063099,0.034149,-0.017550,0.041065,0.005484,0.005868,-0.088957,0.007205,-0.028041,-0.026198,0.099414,-0.034028,-0.038490,-0.008912,0.019575,0.101953,-0.025955,-0.001775,-0.031921,0.054061
7,-0.011033,0.058749,-0.042080,0.032049,-0.018409,0.033911,-0.022273,0.039982,-0.000774,0.001180,0.143314,-0.002568,-0.012992,0.003418,0.065967,0.090875,0.047531,-0.004568,-0.000612,0.191764
8,-0.156186,-0.104851,0.009946,-0.056435,0.003203,0.046723,0.079644,0.005711,0.027830,0.079281,0.074240,0.020693,-0.129355,-0.010430,-0.116462,-0.127231,-0.062822,-0.005007,-0.036097,-0.094672
9,-0.010683,0.103665,0.022979,0.071202,-0.002104,0.002031,0.026421,0.060507,0.044020,0.055025,0.177635,0.085735,-0.024838,-0.013893,0.033666,-0.014461,0.000879,0.058695,0.029302,0.082629


In [0]:
#Get the monthly return of Oct-2008
df_r_Oct_2008 = df_monthly_r_full_data.iloc[[-2]].copy()
#Get the monthly return of Nov-2008
df_r_Nov_2008 = df_monthly_r_full_data.iloc[[-1]].copy()
#Get the monthly return from Jan-2005 to Sep-2008
df_monthly_r_sample = df_monthly_r_full_data.iloc[:-2,:]
df_monthly_r_sample

,F,CAT,DIS,MCD,KO,PEP,WMT,C,WFC,JPM,AAPL,IBM,PFE,JNJ,XOM,MRO,ED,T,VZ,NEM
0,-0.032168,0.076198,-0.024101,0.021303,0.031573,0.002980,-0.015077,-0.027115,-0.031321,-0.012317,0.166710,-0.008991,0.088163,0.013910,0.226938,0.222308,-0.025530,0.025628,0.021605,0.081991
1,-0.104348,-0.037980,0.028275,-0.058646,-0.026401,-0.015411,-0.029064,-0.057422,0.022963,-0.053352,-0.071110,-0.011079,0.006725,0.028212,-0.053990,0.013414,-0.000560,-0.015378,-0.013067,-0.061111
2,-0.195940,-0.037073,-0.081100,-0.058767,0.056210,0.053719,-0.056496,0.044949,0.002341,0.025723,-0.134629,-0.164150,0.034259,0.021888,-0.043121,-0.007459,0.026079,0.004643,0.008451,-0.099300
3,0.107228,0.078873,0.039394,0.055612,0.027394,0.011143,0.001910,0.004133,0.007841,0.017426,0.102607,-0.010866,0.026868,-0.022293,-0.014554,0.041228,0.051525,-0.004379,0.000869,-0.019226
4,0.026053,0.012751,-0.082361,-0.103103,-0.064530,-0.041415,0.023805,-0.018680,0.035862,-0.012028,-0.074195,-0.015261,-0.004633,-0.026548,0.027806,0.123911,0.042313,0.015825,-0.023459,0.050946
5,0.048828,0.131256,0.018268,0.123244,0.061672,0.015850,0.023859,-0.059053,-0.003897,-0.005097,0.158652,0.124798,-0.039159,-0.016000,0.022272,0.093498,0.028181,0.029474,-0.009262,-0.037919
6,-0.063099,0.034149,-0.017550,0.041065,0.005484,0.005868,-0.088957,0.007205,-0.028041,-0.026198,0.099414,-0.034028,-0.038490,-0.008912,0.019575,0.101953,-0.025955,-0.001775,-0.031921,0.054061
7,-0.011033,0.058749,-0.042080,0.032049,-0.018409,0.033911,-0.022273,0.039982,-0.000774,0.001180,0.143314,-0.002568,-0.012992,0.003418,0.065967,0.090875,0.047531,-0.004568,-0.000612,0.191764
8,-0.156186,-0.104851,0.009946,-0.056435,0.003203,0.046723,0.079644,0.005711,0.027830,0.079281,0.074240,0.020693,-0.129355,-0.010430,-0.116462,-0.127231,-0.062822,-0.005007,-0.036097,-0.094672
9,-0.010683,0.103665,0.022979,0.071202,-0.002104,0.002031,0.026421,0.060507,0.044020,0.055025,0.177635,0.085735,-0.024838,-0.013893,0.033666,-0.014461,0.000879,0.058695,0.029302,0.082629


## 2.0 Parameter Estimation

### 2.1 Estimate the sample mean

In [0]:
#compute the sample mean for each stock (n=45)
r_sample_means = df_monthly_r_sample.mean(axis = 0) 
display(r_sample_means)

F      -0.024767
CAT     0.002276
DIS     0.000021
MCD     0.016395
KO      0.006798
PEP     0.004174
WMT     0.003744
C      -0.023051
WFC     0.009118
JPM     0.007839
AAPL    0.031563
IBM     0.003081
PFE    -0.001950
JNJ     0.001406
XOM     0.011410
MRO     0.019815
ED      0.004677
T       0.008135
VZ      0.002234
NEM    -0.004393
dtype: float64

### 2.2 Compute the sample variance

In [0]:
#sample variance for each stock
r_sample_std = df_monthly_r_sample.std(axis = 0, ddof=1)
r_sample_var = r_sample_std**2
display(r_sample_var) 

F       0.021957
CAT     0.007196
DIS     0.002523
MCD     0.002818
KO      0.002031
PEP     0.002249
WMT     0.002129
C       0.008495
WFC     0.005702
JPM     0.005632
AAPL    0.016221
IBM     0.004144
PFE     0.002892
JNJ     0.001277
XOM     0.003762
MRO     0.011952
ED      0.001452
T       0.003348
VZ      0.002610
NEM     0.009671
dtype: float64

### 2.3 Compute the sample covariance

In [0]:
#compute sample covariances
r_sample_cov = np.cov(df_monthly_r_sample.T)

display(r_sample_cov)

array([[ 2.19571849e-02,  5.98011908e-03,  3.26562149e-03,
         3.19291322e-03,  2.75749820e-03,  2.27921991e-03,
         2.74777161e-03,  8.78443030e-03,  3.15327152e-03,
         4.94759276e-03,  2.88076734e-03,  4.67733099e-03,
         1.58508508e-03,  1.85570584e-03,  2.94001642e-03,
         4.39637320e-03,  1.00835062e-03,  1.64008719e-03,
         2.44857244e-03,  3.68185030e-03],
       [ 5.98011908e-03,  7.19551082e-03,  2.32417253e-03,
         1.83091738e-03,  1.78325199e-03,  1.65086433e-03,
         3.79189396e-04,  2.92681749e-03, -2.40942317e-04,
         6.39006455e-04,  4.75301202e-03,  2.54405801e-03,
         1.19077051e-03,  6.81283528e-04,  2.19196643e-03,
         5.22967383e-03, -6.42331534e-05,  2.22480224e-03,
         2.10347191e-03,  3.90776114e-03],
       [ 3.26562149e-03,  2.32417253e-03,  2.52276056e-03,
         1.11835215e-03,  1.10032479e-03,  1.05358013e-03,
         4.31567342e-04,  1.61271023e-03, -2.69939317e-04,
         1.74696983e-04,  1.5

In [0]:
r_sample_cov.shape

(20, 20)

## 2.3 Compute the market portfolio (P4)

### 2.3.1 Get the shares outstanding of each stock as of September 2008

In [0]:
#Compute estimated market portfolio

#Searched shares outstandingg of the 20 stocks as of Sepetember 2008:

shares_outstanding = {
    'F':2.279,
    'CAT':0.625,
    'DIS':1.948,
    'MCD':1.136,
    'KO':4.658,
    'PEP':1.593,
    'WMT':3.958,
    'C':0.583,
    'WFC':3.331,
    'JPM':3.445,
    'AAPL':6.315,
    'IBM':1.384,
    'PFE':6.736,
    'JNJ':2.831,
    'XOM':5.178,
    'MRO':0.711,
    'ED':0.274,
    'T':5.893,
    'VZ':2.845,
    'NEM':0.455
}

In [0]:
pd_shares_outstanding = pd.Series(shares_outstanding)
display(pd_shares_outstanding)

F       2.279
CAT     0.625
DIS     1.948
MCD     1.136
KO      4.658
PEP     1.593
WMT     3.958
C       0.583
WFC     3.331
JPM     3.445
AAPL    6.315
IBM     1.384
PFE     6.736
JNJ     2.831
XOM     5.178
MRO     0.711
ED      0.274
T       5.893
VZ      2.845
NEM     0.455
dtype: float64

### 2.3.2 Compute the market capitalization

In [0]:
market_capital = df_stock_price_full.iloc[-3,1:]*pd_shares_outstanding
market_capital


F       3.50817
CAT     16.6066
DIS     42.8201
MCD     45.7619
KO      63.7062
PEP     64.0448
WMT     166.495
C       72.0842
WFC     81.8628
JPM     106.498
AAPL    84.0259
IBM     89.7189
PFE     75.4601
JNJ     121.656
XOM     261.075
MRO     9.05964
ED      7.21156
T       81.7693
VZ      44.2653
NEM     9.90039
dtype: object

### 2.3.3 Compute Market portfolio (Assume the 20 stocks are the all market size)

In [0]:
total_mrkt = market_capital.sum()
x_p4_mrkt = market_capital/total_mrkt
x_p4_mrkt

F       0.00242356
CAT      0.0114724
DIS      0.0295815
MCD      0.0316138
KO       0.0440103
PEP      0.0442442
WMT        0.11502
C        0.0497981
WFC      0.0565534
JPM       0.073572
AAPL     0.0580478
IBM      0.0619807
PFE      0.0521303
JNJ      0.0840438
XOM       0.180359
MRO     0.00625869
ED      0.00498198
T        0.0564889
VZ       0.0305799
NEM     0.00683951
dtype: object

## 2.4 Compute the performance of market portfolio (P4) in October 2008

### 2.4.1 Compute the realized portfolio return of market portfolio (P4) in October 2008

In [0]:
#calculate P market realized return rate in Oct 2008

r_oct_p4_mrkt = float(np.dot(df_r_Oct_2008,x_p4_mrkt))
r_oct_p4_mrkt

-0.0431447155993136

### 2.4.2 Compute the realized portfolio variance and standard deviation of market portfolio (P4) in October 2008

In [0]:
r_sample_cov_till_Oct = np.cov(df_monthly_r_full_data.iloc[:-1,:].T)

In [0]:
var_oct_p4_mrkt = np.dot(np.dot(x_p4_mrkt.T,r_sample_cov_till_Oct),x_p4_mrkt)
var_oct_p4_mrkt

0.0011814047238156858

In [0]:
std_oct_p4_mrkt = np.sqrt(var_oct_p4_mrkt)
std_oct_p4_mrkt

0.03437156853877469

### 2.4.3 Compute the realized portfolio sharp ratio of market portfolio (P4) in October 2008

#### 2.4.3.1 risk free rate
https://www.macrotrends.net/2016/10-year-treasury-bond-rate-yield-chart


In [0]:
rf_month = np.power(1+0.036,1/12)-1
rf_month

0.0029516094330215292

In [0]:
sharp_oct_p4_mrkt = float((r_oct_p4_mrkt-rf_month)/std_oct_p4_mrkt)
sharp_oct_p4_mrkt

-1.3411178771295729

## 2.5 Compute the portfolio based on MVO (P1)

### 2.5.1 compute λ

In [0]:
#calculate market expected return rate
r_exp_mrkt = np.dot(x_p4_mrkt,r_sample_means)
r_exp_mrkt

0.0060865585680787435

In [0]:
#calculate market expected variance
x_mrkt = np.array(x_p4_mrkt).reshape(20,1)
var_exp_mrkt = float(np.dot(np.dot(x_mrkt.T,r_sample_cov),x_mrkt))
var_exp_mrkt

0.0011543678123873253

In [0]:
#Risk aversion coefficient lambda
λ = (r_exp_mrkt-rf_month)/var_exp_mrkt
λ

2.715728125313792

###2.5.2 compute x of Portfolio 1

https://pypi.org/project/qpsolvers/


https://scaron.info/blog/quadratic-programming-in-python.html


In [0]:
r_sample_means

F      -0.024767
CAT     0.002276
DIS     0.000021
MCD     0.016395
KO      0.006798
PEP     0.004174
WMT     0.003744
C      -0.023051
WFC     0.009118
JPM     0.007839
AAPL    0.031563
IBM     0.003081
PFE    -0.001950
JNJ     0.001406
XOM     0.011410
MRO     0.019815
ED      0.004677
T       0.008135
VZ      0.002234
NEM    -0.004393
dtype: float64

In [0]:
f = -1*np.array(r_sample_means)
print(f.shape)
print(f)

(20,)
[ 2.47672181e-02 -2.27597375e-03 -2.06740796e-05 -1.63954823e-02
 -6.79777556e-03 -4.17430824e-03 -3.74375358e-03  2.30509605e-02
 -9.11769936e-03 -7.83930154e-03 -3.15632374e-02 -3.08112933e-03
  1.94988622e-03 -1.40590012e-03 -1.14095467e-02 -1.98145068e-02
 -4.67715734e-03 -8.13488780e-03 -2.23365579e-03  4.39310731e-03]


In [0]:
A = np.zeros(20)
print(A.shape)
print(A)

(20,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [0]:
b = 0.
print(b)

0.0


In [0]:
Aeq = np.ones(20)
print(Aeq.shape)
print(Aeq)

(20,)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [0]:
beq = np.array([1.])
print(beq.shape)
print(beq)

(1,)
[1.]


In [0]:
Q = 2*λ*r_sample_cov
print(Q.shape)

(20, 20)


In [0]:
np.all(np.linalg.eigvals(Q) > 0)

True

In [0]:
from numpy import array, dot
from qpsolvers import solve_qp
x_p1_mvo = solve_qp(Q, f, A, b, Aeq, beq, solver='quadprog')
print ("QP solution or x_mvo:", x_p_MVO)

QP solution or x_mvo: [ 0.07942287 -0.23390853  1.08976222  1.24489521 -0.08250134  0.41608393
 -0.25600045 -2.4240574   3.0797168   0.54151626  0.41419343 -1.64107293
 -2.33264349 -0.01777243  0.82519546  0.82983423 -1.32067097  1.03393147
 -0.27827866  0.03235432]


In [0]:
r_exp_p1_MVO = np.dot(r_sample_means,x_p1_mvo)
r_exp_p1_MVO

0.14622166969587586

### 2.5.3 Compute the realized portfolio return of MVO portfolio (P1) in October 2008

In [0]:
r_oct_p1_MVO = float(np.dot(df_r_Oct_2008,x_p_MVO))
r_oct_p1_MVO

0.6744859763019845

### 2.5.4 Compute the realized portfolio variance and standard deviation of MVO portfolio (P1) in October 2008

In [0]:
var_oct_p1_mvo = np.dot(np.dot(x_p1_mvo.T,r_sample_cov_till_Oct),x_p1_mvo)
var_oct_p1_mvo

0.031622592547295866

In [0]:
std_oct_p1_mvo = np.sqrt(var_oct_p1_mvo)
std_oct_p1_mvo

0.1778274234961972

### 2.5.5 Compute the realized portfolio sharp ratio of MVO portfolio (P1) in October 2008

In [0]:
sharp_oct_p1_mvo = float((r_oct_p1_MVO-rf_month)/std_oct_p1_mvo)
sharp_oct_p1_mvo

3.7763262474717436

## 2.6 Compute the portfolio based on Robust MVO (P2)

### 2.6.1 Robust MVO with confinence level 0f 90% (P2_90)

In [0]:
chi_90 = 28.41

In [0]:
T = 45 #as we have 45 sample observations
theta = np.diag(np.diag(r_sample_cov))/T
sqrt_theta = np.sqrt(theta)

In [0]:
def opt_func(x,ep):
  return λ*np.dot(np.dot(x.T,r_sample_cov),x)-np.dot(r_sample_means,x)+ep*np.linalg.norm(np.dot(sqrt_theta,x))

In [0]:
from scipy.optimize import LinearConstraint

#define the constraint of e*x = 1
linear_constraint = LinearConstraint([np.ones(20)],[1],[1])

In [0]:
#initial x
x0 = np.array([1/20]*20)
x0

array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
       0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])

In [0]:
from scipy.optimize import minimize

x_p2_RMVO = minimize(lambda x:opt_func(x,chi_90),x0,method='trust-constr',constraints=linear_constraint)

x_p2_RMVO

         cg_niter: 80
     cg_stop_cond: 4
           constr: [array([1.])]
      constr_nfev: [0]
      constr_nhev: [0]
      constr_njev: [0]
   constr_penalty: 1.0
 constr_violation: 0.0
   execution_time: 0.16249585151672363
              fun: 0.05192676433679508
             grad: array([0.05422563, 0.05422562, 0.05422563, 0.05422562, 0.05422563,
       0.05422562, 0.05422562, 0.05422563, 0.05422563, 0.05422562,
       0.05422563, 0.05422562, 0.05422562, 0.05422563, 0.05422562,
       0.05422562, 0.05422563, 0.05422563, 0.05422563, 0.05422563])
              jac: [array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.]])]
  lagrangian_grad: array([ 1.51339920e-09, -1.74622981e-09,  1.51339920e-09, -8.14907239e-10,
        3.84170563e-09, -1.28056853e-09, -2.21189110e-09,  1.04773791e-09,
        3.37604435e-09, -2.67755239e-09,  1.51339920e-09, -1.28056853e-09,
       -1.28056853e-09,  1.16415336e-10, -8.14907239e-10, -7.79982655e-09,
     

In [0]:
optx = x_p2_RMVO.x
optx

array([0.00207529, 0.0204243 , 0.06022695, 0.06977498, 0.08358788,
       0.07237071, 0.08106483, 0.00770503, 0.03203958, 0.03042164,
       0.01417538, 0.03774146, 0.05162568, 0.1241721 , 0.04978021,
       0.01699058, 0.11874368, 0.05221247, 0.06015992, 0.01470731])

In [0]:
optx.sum()

1.0

In [0]:
r_exp_p2_RMVO = np.dot(r_sample_means,optx)
r_exp_p2_RMVO

0.005259262188784121

In [0]:
r_oct_p2_RMVO = float(np.dot(df_r_Oct_2008,optx))
r_oct_p2_RMVO

-0.02038251400428203

# For test only of using Scipy.minimize to solve MVO

In [0]:
def opt_func_mvo(x):
  return -np.dot(r_sample_means,x)+λ*np.dot(np.dot(x.T,r_sample_cov),x)

In [0]:
#define the constraint of e*x = 1
linear_constraint_mvo = LinearConstraint([np.ones(20)],[1],[1])

In [0]:
opt_MVO_solution = minimize(lambda x:opt_func_mvo(x),x0,method='trust-constr',constraints=linear_constraint_mvo)

In [0]:
x_opt_mvo= opt_MVO_solution.x
x_opt_mvo

array([ 0.07942278, -0.23390866,  1.08976341,  1.24489494, -0.08250207,
        0.41608349, -0.25600095, -2.42405773,  3.07971685,  0.54151714,
        0.41419332, -1.64107312, -2.33264393, -0.0177719 ,  0.82519612,
        0.82983428, -1.32067124,  1.03393169, -0.27827879,  0.03235438])

In [0]:
r_opt_mvo = np.dot(r_sample_means,x_opt_mvo)
r_opt_mvo

0.14622167933283883

In [0]:
r_opt_oct_mvo = float(np.dot(df_r_Oct_2008,x_opt_mvo))
r_opt_oct_mvo

0.6744857913293575

### 2.6.2 Robust MVO with confinence level 0f 95% (P2_95)

In [0]:
chi_95 = 31.41